In [ ]:
from keras import applications
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model,Input
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout,Reshape
from keras.optimizers import SGD,Adam
import keras.backend as K

In [ ]:
from __future__ import print_function
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten,Activation, Average,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.callbacks import CSVLogger

In [ ]:
def DenseNet121(model_input):
    base_model = applications.densenet.DenseNet121(weights='imagenet', include_top=False, input_shape=(227,227, 3),input_tensor=model_input)
    last = base_model.output
    x = Flatten()(last)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(28, activation='relu')(x)
    preds = Reshape((2,14))(x)
    model = Model(base_model.input, preds)
    return model

In [ ]:
def ResNet50(model_input):
    base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(227,227, 3),input_tensor=model_input)
    last = base_model.output
    x = Flatten()(last)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(28, activation='relu')(x)
    preds = Reshape((2,14))(x)
    model = Model(base_model.input, preds)
    return model

In [ ]:
model_input = Input(shape=(227,227,3))

In [ ]:
densenet_model = ResNet50_2(model_input)

In [ ]:
resnet_model = ResNet50(model_input)

In [ ]:
resnet_model.load_weights('resnet50_t05_lr00005_v2.h5')
densenet_model.load_weights('densenet121_t05_lr00005.h5')
ensembled_models = [resnet_model,densenet_model]

In [ ]:
def ensemble(models,model_input):
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    model = Model(model_input, y, name='ensemble')
    return model

In [ ]:
ensemble_model = ensemble(ensembled_models,model_input)

In [ ]:
# custom loss function (L2 distance)
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true)))/64

In [ ]:
thresh = 0.5
def accuracy(y_true, y_pred):
    
    diff = K.square(y_true - y_pred)
#     loss = K.sum(diff)
    dist = K.sqrt(K.sum(diff, axis = 1))	# (N, 2, 14) -> (N, 14)
#     accuracy = tf.reduce_sum(tf.to_int32(tf.greater(1.0, dist))) / 8/14
    torsor_xy = y_true[:,:,9] - y_true[:,:,2] # (N,2)
    torsor_dist = K.sqrt(K.sum(K.square(torsor_xy),axis=1,keepdims=True)) #(N,1)
    torsor_frac = thresh*torsor_dist
    
    accuracy = K.sum(K.cast(K.greater(torsor_frac,dist),dtype='float32'))/64/14
    
    return accuracy

In [ ]:
# compile the model
ensemble_model.compile(optimizer=Adam(lr=0.00005), loss=euclidean_distance_loss, metrics=[accuracy])

In [ ]:
from parse_lsp_data import *
import tensorflow as tf

In [ ]:
# get the data set
train_list, train_label, val_list, val_label, test_list, test_label = getLSPDataset(0.8, 0.1)

In [ ]:
# print some information
train_X = train_list	# (N, 227, 227, 3)
train_Y = train_label	# (N, 2, 14)
print("train X: ", train_X.shape, "val X: ", val_list.shape, "test X: ", test_list.shape)
print("train Y: ", train_Y.shape, "val Y: ", val_label.shape, "test Y: ", test_label.shape)

In [ ]:
# evaluation
with tf.device('/device:GPU:0'):
    loss,acc = ensemble_model.evaluate(test_list, test_label,batch_size=64)

In [ ]:
print(loss)
print(acc)

In [ ]:
pred = ensemble_model.predict(test_list)

In [ ]:
from drawLines_v2 import drawLines
# usage:
index = 6
drawLines(test_list[index].copy(),test_label[index].copy(),pred[index].copy())